# **Marker Position prediction from IMU data using deep learning.**

As a part of the Article: 
DOI: (Will be added)

# Author: Vaibhav R. Shah and Philippe C. Dixon

We recomand you run on jupyter notebook

## Import required libraries

In [1]:
import sys
import numpy as np
import GaitLab2Go as GL2G
import marker_prediction as mp
import os
import warnings

# Suppress all warnings
warnings.filterwarnings("ignore")

## Processing data

In [2]:
import GaitLab2Go as GL2G
lab=GL2G.data_processing()
fld= os.path.join(".","data", "pp054", "imu")
ext='.zoo'

#Converting .zoo files to .pkl files (Compatible with GaitLab2Go library)
mp.convert_zoo2pickle()

# Assign the subject list to the variable 'subject'
subject_list = ['pp054']
fld= os.path.join(".","data", "pp054", "imu")
ext='.pkl'
variable = lab.variables_zoo_IMU()
# Assign the file list to 'files' (previously found using lab.find_files)
files = lab.find_files(path=fld, ext=ext) # Replace with your actual file list if needed

# Call the process_subject_data_task function with the subject list, file list, and variable list
result_data = mp.process_subject_data_task(subject_list, files, variable)


mp.extracting_markerwise_data(result_data)


Extracting data from .\data\pp054\imu\pp054_WT_01_cycle_1.zoo
Extracting complete .\data\pp054\imu\pp054_WT_01_cycle_1.zoo
saving as pickle file to .\data\pp054\imu\pp054_WT_01_cycle_1.pkl
Extracting data from .\data\pp054\imu\pp054_WT_01_cycle_2.zoo
Extracting complete .\data\pp054\imu\pp054_WT_01_cycle_2.zoo
saving as pickle file to .\data\pp054\imu\pp054_WT_01_cycle_2.pkl
['pp054']
['.\\data\\pp054\\imu\\pp054_WT_01_cycle_1.pkl'
 '.\\data\\pp054\\imu\\pp054_WT_01_cycle_2.pkl']
saving file to .\data\pp054\imu\pp054_WT_01_cycle_1.pkl
saving file to .\data\pp054\imu\pp054_WT_01_cycle_2.pkl
.\data\pp054\imu\pp054_WT_01_cycle_1.pkl
walk
.\data\pp054\imu\pp054_WT_01_cycle_2.pkl
walk
Saving data for LASI
Saving data for RASI
Saving data for LPSI
Saving data for RPSI
Saving data for LTHI
Saving data for LKNE
Saving data for LTIB
Saving data for LANK
Saving data for LHEE
Saving data for LTOE
Saving data for RTHI
Saving data for RKNE
Saving data for RTIB
Saving data for RANK
Saving data for R

# Testing deep learning Models

In [3]:
data_LASI, train=mp.loading_predition_data(fld = os.path.join(".","data", "pp054", "predicted_markers"))


fld=os.path.join(".","data", "pp054", "predicted_markers")
# List of participants (can be extended to include more)
participants = ['pp054']
Select_subject=['pp054']
print("Prediction Start")
participants_data = mp.predict_marker_position(fld,participants,data_LASI, train, Select_subject)
print("Prediction Complete")

fld=os.path.join(".","data", "pp054", "predicted_markers")
mp.average_prediction(fld,participants_data,participants)

Loading IMU data for prediction
Combining IMU data for prediction
Converting IMU data Units to required units
Angular velocity of gyroscope converted to radian/s from deg/s
Linear acceleration data in m/s from g by multiplying with 9.81
Rotating sensors to reorient gyroscope and accelerometer data
Prediction Start
1/1 ━━━━━━━━━━━━━━━━━━━━ 7s 7s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 6s 6s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 6s 6s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 5s 5s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 6s 6s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 6s 6s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 5s 5s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 5s 5s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 6s 6s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 7s 7s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 6s 6s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 6s 6s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 5s 5s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 7s 7s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 7s 7s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 5s 5s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 5s 5s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 5s 5s/step
Prediction Complete
Saving the avera

# Visualizing predicted marker positions

**Walking Stick Skeleton** 

In [4]:
import matplotlib.pyplot as plt
# Set an interactive backend
%matplotlib qt5
mp.visulize_predicted_data()